In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

In [38]:
data_type = {
    '_CustomerID': str 
}

In [39]:
sales_data = pd.read_csv('sales_data.csv', dtype = data_type, parse_dates=['OrderDate'])

In [40]:
sales_data.head()

,OrderNumber,Sales Channel,WarehouseCode,ProcuredDate,OrderDate,ShipDate,DeliveryDate,CurrencyCode,_SalesTeamID,_CustomerID,_StoreID,_ProductID,Order Quantity,Discount Applied,Unit Price,Unit Cost
0,SO - 000101,In-Store,WARE-UHY1004,12/31/2017,2018-05-31,6/14/2018,6/19/2018,USD,6,15,259,12,5,0.075,1963.1,1001.181
1,SO - 000102,Online,WARE-NMK1003,12/31/2017,2018-05-31,6/22/2018,7/2/2018,USD,14,20,196,27,3,0.075,3939.6,3348.660
2,SO - 000103,Distributor,WARE-UHY1004,12/31/2017,2018-05-31,6/21/2018,7/1/2018,USD,21,16,213,16,1,0.050,1775.5,781.220
3,SO - 000104,Wholesale,WARE-NMK1003,12/31/2017,2018-05-31,6/2/2018,6/7/2018,USD,28,48,107,23,8,0.075,2324.9,1464.687
4,SO - 000105,Distributor,WARE-NMK1003,4/10/2018,2018-05-31,6/16/2018,6/26/2018,USD,22,49,111,26,8,0.100,1822.4,1476.144


In [41]:
df = sales_data.copy()

In [62]:
df['Revenue'] = (df['Unit Price'] - df['Unit Cost']) * (1 - df['Discount Applied']) * df['Order Quantity']

In [43]:
df.dtypes

OrderNumber                 object
Sales Channel               object
WarehouseCode               object
ProcuredDate                object
OrderDate           datetime64[ns]
ShipDate                    object
DeliveryDate                object
CurrencyCode                object
_SalesTeamID                 int64
_CustomerID                 object
_StoreID                     int64
_ProductID                   int64
Order Quantity               int64
Discount Applied           float64
Unit Price                 float64
Unit Cost                  float64
Revenue                    float64
dtype: object

In [63]:
columns_to_use = ['_CustomerID', 'OrderDate', 'Revenue', 'OrderNumber']

In [64]:
df2 = df[columns_to_use]

In [74]:
max_date =  pd.to_datetime('2020-12-30')

In [75]:
RFM_dataset = df2.groupby('_CustomerID').agg({
    'OrderDate': lambda x: (max_date - x.max()).days,
    'OrderNumber': 'count',
    'Revenue': 'sum'
    }).reset_index()

In [76]:
RFM_dataset

,_CustomerID,OrderDate,OrderNumber,Revenue
0,1,7,152,432755.708475
1,10,13,158,555864.221975
2,11,4,178,627510.001975
3,12,1,210,765837.632475
4,13,2,171,569885.362225
5,14,3,157,494928.767175
6,15,2,142,539306.233925
7,16,1,135,514704.922675
8,17,4,175,652958.911800
9,18,4,186,590287.162050


In [77]:
RFM_dataset.rename(
    columns={
    'OrderDate': 'Recency',
    'OrderNumber': 'Frequency',
    'Revenue': 'Monetary'
}), 
inplace=True

